# Ad-Hoc Analyses
- **Project:** GP2 AFR-AAC meta-GWAS 
- **Version:** Mix of bash and R
- **Status:** COMPLETE
- **Started:** 02-MAY-2023
- **Last Updated:** 02-MAY-2023
    - **Update Description:** Adding ad-hoc analyses to one notebook

## Notebook Overview
- Investigating runs of homozygosity, linear regression for % admixture and age, haplotypes with 1KG, fine-mapping, dominant vs recessive modeling, annotating WGS for T2, conditional analysis, Miami plot compared to Europeans, and GBA Gauchian caller WDL

### CHANGELOG
- 02-MAY-2023: Notebook started 

---
# Data Overview 

| ANCESTRY |     DATASET     | CASES | CONTROLS |  TOTAL  |           ARRAY           |                NOTES                |
|:--------:|:---------------:|:-----:|:--------:|:-------------------------:|:---------------------------------------------------------------------------------------------------------------:|:-----------------------------------:|
|    AFR   | IPDGC – Nigeria |  304  |    285   |   589   |         NeuroChip         | . | 
|    AFR   |  GP2  |  711  |   1,011  |  1,722  |        NeuroBooster       | . |
|    AAC   |  GP2 |  185  |   1,149  |  1,334  |        NeuroBooster       | . | 
|    AAC   |     23andMe     |  288  |  193,985 | 194,273 | Omni Express & GSA & 550k |        Just summary statistics       |

# Getting Started

# Runs of Homozygosity

```bash
## ROH in GP2 AFR - NeuroBooster data

mkdir ROH

plink --bfile UPDATED-GP2-v3-AFR-wNIGERIAN-NB \
--indep-pairwise 50 5 0.5 \
--out ROH/pruning

cd ROH

plink --bfile UPDATED-GP2-v3-AFR-wNIGERIAN-NB \
--extract pruning.prune.in \
--make-bed --out pruned_data

plink --bfile pruned_data \
--homozyg \
--homozyg-density 50 \
--homozyg-gap 1000 \
--homozyg-kb 1000 \
--homozyg-snp 10 \
--homozyg-window-het 1 \
--homozyg-window-missing 10 \
--homozyg-window-snp 20 \
--homozyg-window-threshold 0.05 \
--out pruned_ROH_UPDATED-GP2-v3-AFR-wNIGERIAN-NB

# only chr 1
plink --bfile pruned_data \
--chr 1 \
--homozyg \
--homozyg-density 50 \
--homozyg-gap 1000 \
--homozyg-kb 1000 \
--homozyg-snp 10 \
--homozyg-window-het 1 \
--homozyg-window-missing 10 \
--homozyg-window-snp 20 \
--homozyg-window-threshold 0.05 \
--out pruned_ROH_UPDATED-GP2-v3-AFR-wNIGERIAN-NB_chr1
```

```R
## Explore NSEG, KB and KBAVG differences in cases versus controls
data <- read.table("pruned_ROH_UPDATED-GP2-v3-AFR-wNIGERIAN-NB.hom.indiv", header = T)
head(data)
data$case <- data$PHE - 1
model1 <- glm(case ~ NSEG, data = data, family = "binomial")
summary(data$case)
dat <- subset(data, case != -10)
length(dat$case)
model1 <- glm(case ~ NSEG, data = dat, family = "binomial")
summary(model1)
```

```
# Call:
# glm(formula = case ~ NSEG, family = "binomial", data = dat)
# 
# Deviance Residuals: 
#     Min       1Q   Median       3Q      Max  
# -2.4855  -1.0001  -0.8484   1.2944   1.6926  
# 
# Coefficients:
#             Estimate Std. Error z value Pr(>|z|)    
# (Intercept) -2.12945    0.23385  -9.106  < 2e-16 ***
# NSEG         0.08081    0.01052   7.684 1.54e-14 ***
# ---
# Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
# 
# (Dispersion parameter for binomial family taken to be 1)
# 
#     Null deviance: 2300.5  on 1701  degrees of freedom
# Residual deviance: 2232.0  on 1700  degrees of freedom
# AIC: 2236
# 
# Number of Fisher Scoring iterations: 4
```

```R
model2 <- glm(case ~ KB, data = dat, family = "binomial")
summary(model2)
```

```
# Call:
# glm(formula = case ~ KB, family = "binomial", data = dat)
# 
# Deviance Residuals: 
#     Min       1Q   Median       3Q      Max  
# -2.6162  -1.0014  -0.9498   1.3290   1.4806  
# 
# Coefficients:
#               Estimate Std. Error z value Pr(>|z|)    
# (Intercept) -1.006e+00  1.444e-01  -6.969 3.19e-12 ***
# KB           1.593e-05  3.466e-06   4.596 4.31e-06 ***
# ---
# Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
# 
# (Dispersion parameter for binomial family taken to be 1)
# 
#     Null deviance: 2300.5  on 1701  degrees of freedom
# Residual deviance: 2271.6  on 1700  degrees of freedom
# AIC: 2275.6
# 
# Number of Fisher Scoring iterations: 4

# model3 <- glm(case ~ KBAVG, data = dat, family = "binomial")
# summary(model3)
# 
# Call:
# glm(formula = case ~ KBAVG, family = "binomial", data = dat)
# 
# Deviance Residuals: 
#     Min       1Q   Median       3Q      Max  
# -1.4197  -1.0174  -0.9996   1.3410   1.3939  
# 
# Coefficients:
#               Estimate Std. Error z value Pr(>|z|)   
# (Intercept) -0.8328821  0.2534119  -3.287  0.00101 **
# KBAVG        0.0002524  0.0001372   1.840  0.06573 . 
# ---
# Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
# 
# (Dispersion parameter for binomial family taken to be 1)
# 
#     Null deviance: 2300.5  on 1701  degrees of freedom
# Residual deviance: 2297.0  on 1700  degrees of freedom
# AIC: 2301
# 
# Number of Fisher Scoring iterations: 4
```

```bash
## ROH in GP2 AAC - Neurobooster data
cd /data/GP2/projects/2023_01_MBM_AFR_AAC_GWAS/data/AAC/

mkdir ROH

plink --bfile UPDATED-GP2-v3-AAC/UPDATED-GP2-v3-AAC \
--indep-pairwise 50 5 0.5 \
--out ROH/pruning

cd ROH

plink --bfile /data/GP2/projects/2023_01_MBM_AFR_AAC_GWAS/data/AAC/UPDATED-GP2-v3-AAC/UPDATED-GP2-v3-AAC \
--extract pruning.prune.in \
--make-bed --out pruned_data

plink --bfile pruned_data \
--homozyg \
--homozyg-density 50 \
--homozyg-gap 1000 \
--homozyg-kb 1000 \
--homozyg-snp 10 \
--homozyg-window-het 1 \
--homozyg-window-missing 10 \
--homozyg-window-snp 20 \
--homozyg-window-threshold 0.05 \
--out pruned_ROH_UPDATED-GP2-v3-AAC

# only chr 1
plink --bfile pruned_data \
--chr 1 \
--homozyg \
--homozyg-density 50 \
--homozyg-gap 1000 \
--homozyg-kb 1000 \
--homozyg-snp 10 \
--homozyg-window-het 1 \
--homozyg-window-missing 10 \
--homozyg-window-snp 20 \
--homozyg-window-threshold 0.05 \
--out pruned_ROH_UPDATED-GP2-v3-AAC_chr1
```

```R
## Explore NSEG, KB and KBAVG differences in cases versus controls
data <- read.table("pruned_ROH_UPDATED-GP2-v3-AAC.hom.indiv", header = T)
head(data)
data$case <- data$PHE - 1
dat <- subset(data, case != -10)
model1 <- glm(case ~ NSEG, data = dat, family = "binomial")
summary(model1)
```

```
# Call:
# glm(formula = case ~ NSEG, family = "binomial", data = dat)
# 
# Deviance Residuals: 
#     Min       1Q   Median       3Q      Max  
# -0.7022  -0.5442  -0.5363  -0.5284   2.0326  
# 
# Coefficients:
#             Estimate Std. Error z value Pr(>|z|)    
# (Intercept) -2.02535    0.26761  -7.568 3.78e-14 ***
# NSEG         0.01058    0.01504   0.703    0.482    
# ---
# Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
# 
# (Dispersion parameter for binomial family taken to be 1)
# 
#     Null deviance: 1046.1  on 1313  degrees of freedom
# Residual deviance: 1045.6  on 1312  degrees of freedom
# AIC: 1049.6
# 
# Number of Fisher Scoring iterations: 4
```

```R
model2 <- glm(case ~ KB, data = dat, family = "binomial")
summary(model2)
```

```
# Call:
# glm(formula = case ~ KB, family = "binomial", data = dat)
# 
# Deviance Residuals: 
#     Min       1Q   Median       3Q      Max  
# -0.8099  -0.5408  -0.5370  -0.5335   2.0200  
# 
# Coefficients:
#               Estimate Std. Error z value Pr(>|z|)    
# (Intercept) -1.948e+00  1.252e-01 -15.559   <2e-16 ***
# KB           3.178e-06  2.949e-06   1.078    0.281    
# ---
# Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
# 
# (Dispersion parameter for binomial family taken to be 1)
# 
#     Null deviance: 1046.1  on 1313  degrees of freedom
# Residual deviance: 1045.0  on 1312  degrees of freedom
# AIC: 1049
# 
# Number of Fisher Scoring iterations: 4
```

```R
model3 <- glm(case ~ KBAVG, data = dat, family = "binomial")
summary(model3)
```

```
# Call:
# glm(formula = case ~ KBAVG, family = "binomial", data = dat)
# 
# Deviance Residuals: 
#     Min       1Q   Median       3Q      Max  
# -0.7762  -0.5415  -0.5369  -0.5319   2.0324  
# 
# Coefficients:
#               Estimate Std. Error z value Pr(>|z|)    
# (Intercept) -2.1132342  0.2782441  -7.595 3.08e-14 ***
# KBAVG        0.0001456  0.0001447   1.006    0.314    
# ---
# Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
# 
# (Dispersion parameter for binomial family taken to be 1)
# 
#     Null deviance: 1046.1  on 1313  degrees of freedom
# Residual deviance: 1045.2  on 1312  degrees of freedom
# AIC: 1049.2
# 
# Number of Fisher Scoring iterations: 4
```

# Linear Regression
Age and % admixture

```bash
######## LINEAR REGRESSION VERSUS AGE ########

### AFR
cd GP2-v4-AFR-wNIGERIAN-NB/

# Create age file
awk '{print $1, $2, $20}' masterfile_updated_GP2_v4_covariateFile_wAGE_FEB2023.txt > age_cases.txt

# Filter cases (711 AFR)
plink --bfile GP2-v4-AFR-wNIGERIAN-NB-noRelateds \
--filter-cases \
--make-bed --out Cases_GP2-v4-AFR-wNIGERIAN-NB-noRelateds

# Extract top hit and run linear regression 
plink --bfile Cases_GP2-v4-AFR-wNIGERIAN-NB-noRelateds \
--linear --pheno age_cases.txt --chr 1 --from-bp 155235878 --to-bp 155235878 --covar masterfile_updated_GP2_v4_covariateFile_wAGE_FEB2023.txt --covar-name SEX,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 --out linear_regression

grep "ADD" linear_regression.assoc.linear > results_linear_regression_age_AFR.txt
# (BETA =-2.004, STAT = -3.483, p = 0.0005284)
# BETA/STAT = SE

### AAC
cd AAC/GP2-v4-AAC

# Filter cases (185 AAC)
plink --bfile GP2-v4-AAC-noRelateds --filter-cases --out Cases_GP2-v4-AAC-noRelateds --make-bed

# Extract top hit and run linear regression (only cases)
plink --bfile Cases_GP2-v4-AAC-noRelateds --linear --pheno AFR/GP2-v4-AFR-wNIGERIAN-NB/age_cases.txt --chr 1 --from-bp 155235878 --to-bp 155235878 --covar masterfile_updated_GP2_v4_covariateFile_wAGE_FEB2023.txt --covar-name SEX,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 --out linear_regression

grep "ADD" linear_regression.assoc.linear > results_linear_regression_age_AAC.txt
# (BETA =-4.156 , STAT = -2.443, p = 0.01559)
# BETA/STAT = SE

# Extract HMZ and HETs and run linear regression per genotype
plink --bfile Cases_GP2-v4-AFR-wNIGERIAN-NB-noRelateds --chr 1 --from-bp 155235878 --to-bp 155235878 --recode A --out Cases_GP2-v4-AFR-wNIGERIAN-NB-noRelateds

# HMZ for the risk allele versus HMZ for the non-risk allele 
plink --bfile Cases_GP2-v4-AFR-wNIGERIAN-NB-noRelateds --remove HET_risk.txt --linear --pheno age_cases.txt --chr 1 --from-bp 155235878 --to-bp 155235878 --covar masterfile_updated_GP2_v4_covariateFile_wAGE_FEB2023.txt --covar-name SEX,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 --out linear_regression_HMZ_risk
#  CHR                  SNP         BP   A1       TEST    NMISS       BETA         STAT            P 
#    1   chr1:155235878:G:T  155235878    G        ADD      409      -1.96       -3.062     0.002351

# HET for the risk allele versus HMZ for the non-risk allele 
plink --bfile Cases_GP2-v4-AFR-wNIGERIAN-NB-noRelateds --remove HMZ_risk.txt --linear --pheno age_cases.txt --chr 1 --from-bp 155235878 --to-bp 155235878 --covar masterfile_updated_GP2_v4_covariateFile_wAGE_FEB2023.txt --covar-name SEX,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 --out linear_regression_HET
# CHR                  SNP         BP   A1       TEST    NMISS       BETA         STAT            P 
#    1   chr1:155235878:G:T  155235878    G        ADD      595     -2.285       -2.679     0.007585

# HET for the risk allele versus HMZ for risk allele 
plink --bfile Cases_GP2-v4-AFR-wNIGERIAN-NB-noRelateds --remove HMZ_non_risk.txt --linear --pheno age_cases.txt --chr 1 --from-bp 155235878 --to-bp 155235878 --covar masterfile_updated_GP2_v4_covariateFile_wAGE_FEB2023.txt --covar-name SEX,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 --out linear_regression_HET
#  CHR                  SNP         BP   A1       TEST    NMISS       BETA         STAT            P 
#    1   chr1:155235878:G:T  155235878    T        ADD      370      1.455        1.145       0.2529
```

```R
## Data visualization

library(data.table)
library(ggplot2)
genotypes <- fread("GP2-v4-AFR-wNIGERIAN-NB-noRelateds.rs3115534.raw", header =T) ## here we rename chr1* variant header to "SNP"
covfile <- fread("masterfile_updated_GP2_v4_covariateFile_wAGE_FEB2023.txt", header =T)
data <- merge(genotypes, covfile, by="IID")
data$SNP[data$SNP ==0] <- "TT"
data$SNP[data$SNP==1] <- "GT"
data$SNP[data$SNP ==2] <- "GG"

dat <- subset(data, !is.na(data$SNP))
data <- dat

p <- ggplot(data, aes(x= SNP, y=AAO, fill=as.factor(data$SNP))) + geom_violin(trim=FALSE)
p2 <- p+geom_boxplot(width=0.4, fill="white" ) + theme_minimal()
p2 + scale_fill_manual(values=c("lightblue", "orange", "green")) + theme_bw() + ylab("Age") +xlab("Genotypes") + theme(legend.position = "none")
ggsave("PLOT.jpeg", dpi = 600, units = "in", height = 6, width = 6)
```

```bash         
######## MAF UPDATES ########

plink --bfile GP2-v4-AFR-wNIGERIAN-NB-noRelateds --chr 1 --from-bp 155235878 --to-bp 155235878 --freq --out freqs_AFR
#  CHR                  SNP   A1   A2          MAF  NCHROBS
#   1   chr1:155235878:G:T    G    T       0.2552     3346
   
plink --bfile GP2-v4-AFR-wNIGERIAN-NB-noRelateds --chr 1 --from-bp 155235878 --to-bp 155235878 --assoc --out freqs_AFR
# CHR                  SNP         BP   A1      F_A      F_U   A2        CHISQ            P           OR 
#   1   chr1:155235878:G:T  155235878    G   0.3362   0.1988    T         80.5    2.912e-19        2.042 
   
plink --bfile GP2-v4-AAC-noRelateds --chr 1 --from-bp 155235878 --to-bp 155235878 --freq --out freqs_AAC
#  CHR                  SNP   A1   A2          MAF  NCHROBS
#  1   chr1:155235878:G:T    G    T       0.1485     2646

plink --bfile GP2-v4-AAC-noRelateds --chr 1 --from-bp 155235878 --to-bp 155235878 --assoc --out freqs_AAC
# CHR                  SNP         BP   A1      F_A      F_U   A2        CHISQ            P           OR 
#   1   chr1:155235878:G:T  155235878    G   0.2268    0.136    T        20.57    5.764e-06        1.864 


######## GENOME-WIDE ADMIXTURE AFR PERCENTAGES VERSUS RISK ########

awk '{print $1, $2, $10}' GP2_merge_JAN_24_2023_ready_genotools_callrate_sex_ancestry_merge_train.10.Q.labeled > AFR_AAC_admixture

# ALL cases and controls - looking at FID IID AFRplink --bfile GP2-v4-AFR-wNIGERIAN-NB-noRelateds \
--linear \
--pheno AFR_AAC_admixture \
--chr 1 --from-bp 155235878 --to-bp 155235878 \
--covar masterfile_updated_GP2_v4_covariateFile_wAGE_FEB2023.txt \
--covar-name SEX --out admixture

grep "ADD" admixture.assoc.linear > results_admixture.assoc.linear
#(BETA = 0.0385, STAT=6.031, P=2.002E-9)
# BETA/STAT = SE
```

# Haplotypes with 1KG

SF9 using online 1KG website estimates, but this is an example

```bash
## Calculate Haplotype blocks across AFR subpops using 1000 Genomes data

cd AFR_AAC_NeuroBooster_Chr1

## Packages 
module load plink/1.9

## Keep list of individuals per subpop and generate subsetted plink files (only chr1)
plink --bfile 1kg_30x_GRCh38_chr1 --keep SierraLeone.txt --make-bed --out SIERRALEONE
plink --bfile 1kg_30x_GRCh38_chr1 --keep Esan.txt --make-bed --out ESAN
plink --bfile 1kg_30x_GRCh38_chr1 --keep Gambian.txt --make-bed --out GAMBIAN
plink --bfile 1kg_30x_GRCh38_chr1 --keep Yri.txt --make-bed --out YRI
plink --bfile 1kg_30x_GRCh38_chr1 --keep Kenya.txt --make-bed --out KENYA

## Calculate Hap blocks
plink --bfile ESAN --blocks no-pheno-req --out ESAN
plink --bfile GAMBIAN --blocks no-pheno-req --out GAMBIAN
plink --bfile SIERRALEONE --blocks no-pheno-req --out SIERRALEONE
plink --bfile YRI --blocks no-pheno-req --out YRI
plink --bfile KENYA --blocks no-pheno-req --out KENYA

## Grep Hap spanning risk signal
grep "chr1:155235878" SIERRALEONE.blocks.det > SL
grep "chr1:155235878" ESAN.blocks.det > ESAN
grep "chr1:155235878" GAMBIAN.blocks.det > GAMBIAN
grep "chr1:155235878" YRI.blocks.det > YRI
grep "chr1:155235878" KENYA.blocks.det > KENYA
```

```
# cat SL
#    1    155235878    155247644       11.767     22 chr1:155235878:G:T|chr1:155236550:T:G|chr1:155237031:AT:A|chr1:155237758:T:C|chr1:155237835:G:A|chr1:155237942:C:T|chr1:155238055:T:C|chr1:155238392:T:C|chr1:155239060:T:C|chr1:155239288:G:A|chr1:155239569:C:T|chr1:155240239:G:A|chr1:155240707:T:C|chr1:155240850:A:C|chr1:155242309:A:G|chr1:155242420:A:G|chr1:155242497:C:T|chr1:155243333:T:A|chr1:155244275:C:T|chr1:155244682:A:G|chr1:155245686:C:T|chr1:155247644:AGG:A

# cat ESAN
#    1    155228473    155252386       23.914     51 chr1:155228473:CG:C|chr1:155229524:C:G|chr1:155229875:T:C|chr1:155229883:A:G|chr1:155230420:G:A|chr1:155230533:G:T|chr1:155230588:A:G|chr1:155231273:C:T|chr1:155231399:G:A|chr1:155231444:G:A|chr1:155231912:A:G|chr1:155232202:A:G|chr1:155232898:A:C|chr1:155233269:G:A|chr1:155234893:A:G|chr1:155235540:T:C|chr1:155235878:G:T|chr1:155236550:T:G|chr1:155237031:AT:A|chr1:155237758:T:C|chr1:155237835:G:A|chr1:155237942:C:T|chr1:155238055:T:C|chr1:155238392:T:C|chr1:155239060:T:C|chr1:155239288:G:A|chr1:155239569:C:T|chr1:155240239:G:A|chr1:155240707:T:C|chr1:155240850:A:C|chr1:155242309:A:G|chr1:155242420:A:G|chr1:155242497:C:T|chr1:155243333:T:A|chr1:155244275:C:T|chr1:155244682:A:G|chr1:155245686:C:T|chr1:155247644:AGG:A|chr1:155247647:A:ATT|chr1:155248283:G:A|chr1:155248377:T:C|chr1:155248574:T:C|chr1:155248655:A:G|chr1:155248887:C:G|chr1:155249416:A:T|chr1:155249834:C:A|chr1:155250023:A:G|chr1:155250077:C:A|chr1:155250974:A:G|chr1:155252046:C:A|chr1:155252386:C:T

# cat GAMBIAN
#    1    155235540    155237835        2.296      6 chr1:155235540:T:C|chr1:155235878:G:T|chr1:155236550:T:G|chr1:155237031:AT:A|chr1:155237758:T:C|chr1:155237835:G:A

# cat YRI
#    1    155216938    155258255       41.318     74 chr1:155216938:C:T|chr1:155216951:G:A|chr1:155220463:T:A|chr1:155223032:G:T|chr1:155223741:C:G|chr1:155224349:CAT:C|chr1:155225189:T:C|chr1:155225424:C:G|chr1:155226926:A:G|chr1:155227671:C:T|chr1:155227811:G:C|chr1:155228431:A:G|chr1:155228473:CG:C|chr1:155229524:C:G|chr1:155229875:T:C|chr1:155229883:A:G|chr1:155230420:G:A|chr1:155230533:G:T|chr1:155230588:A:G|chr1:155231273:C:T|chr1:155231399:G:A|chr1:155231444:G:A|chr1:155231912:A:G|chr1:155232202:A:G|chr1:155232898:A:C|chr1:155233269:G:A|chr1:155234893:A:G|chr1:155235540:T:C|chr1:155235878:G:T|chr1:155236550:T:G|chr1:155237031:AT:A|chr1:155237758:T:C|chr1:155237835:G:A|chr1:155237942:C:T|chr1:155238055:T:C|chr1:155238392:T:C|chr1:155239288:G:A|chr1:155239569:C:T|chr1:155240239:G:A|chr1:155240707:T:C|chr1:155240850:A:C|chr1:155242309:A:G|chr1:155242420:A:G|chr1:155242497:C:T|chr1:155243333:T:A|chr1:155244275:C:T|chr1:155244682:A:G|chr1:155245686:C:T|chr1:155247644:AGG:A|chr1:155247647:A:ATT|chr1:155248377:T:C|chr1:155248574:T:C|chr1:155248655:A:G|chr1:155248887:C:G|chr1:155249416:A:T|chr1:155249834:C:A|chr1:155250077:C:A|chr1:155250974:A:G|chr1:155252046:C:A|chr1:155252386:C:T|chr1:155252681:A:T|chr1:155252938:T:C|chr1:155252939:G:A|chr1:155253033:G:A|chr1:155253216:T:A|chr1:155253320:C:T|chr1:155254187:A:G|chr1:155254300:C:A|chr1:155254510:C:A|chr1:155255038:G:C|chr1:155255406:C:G|chr1:155256442:T:C|chr1:155258235:G:A|chr1:155258255:A:G

# cat KENYA
#    1    155235540    155243333        7.794     18 chr1:155235540:T:C|chr1:155235878:G:T|chr1:155236550:T:G|chr1:155237031:AT:A|chr1:155237758:T:C|chr1:155237835:G:A|chr1:155237942:C:T|chr1:155238055:T:C|chr1:155238392:T:C|chr1:155239288:G:A|chr1:155239569:C:T|chr1:155240239:G:A|chr1:155240707:T:C|chr1:155240850:A:C|chr1:155242309:A:G|chr1:155242420:A:G|chr1:155242497:C:T|chr1:155243333:T:A
```

# Fine-mapping

```R
library("data.table")
install.packages("coloc") 
source("https://bioconductor.org/biocLite.R")
biocLite("snpStats")
install.packages("robustbase")
library("robustbase")
install.packages("tidyr")
library(ggplot2)
library(tidyr)
devtools::install_github("chr1swallace/coloc")
library("coloc")
library("tidyverse")

finemap.abf(dataset, p1 = 1e-04)

dataset1 <- fread("GBA_variants.tab", header = T, sep = "\t")
dataset1$type <- "cc"
dataset1$s <- 0.007
dataset1$N <- 196054
dataset_final<- dataset1 %>% mutate(StdErr_squared = StdErr^2)
output <- dataset_final[,c("MarkerName","Effect","P-value","StdErr_squared", "type", "s", "N")]
as.numeric(dataset_final$StdErr_squared)
colnames(output) <- c("SNP","beta","P","varbeta", "type", "s", "N")
fwrite(output, file = "GBA.csv", na = "NA", quote = F, row.names = F, sep = "\t")
results <- finemap.abf(dataset=list(SNP=output$SNP, beta=output$beta, varbeta=output$varbeta, N=196054,s=0.007,type="cc"))
combo <- cbind(results[1:5226,], output)
hits <- subset(combo, SNP.PP > 0.2)
fwrite(combo, file = "GBA_results_fine_map.csv", na = "NA", quote = F, row.names = F, sep = ",")

```

# Dominant vs recessive modeling

```bash
### 1_155235878 dominant model to test G-association in the AAC and AFR cohorts

####### AAC #######
cd GP2-v4-AAC

plink2 --bfile GP2-v4-AAC-noRelateds \
--glm dominant --chr 1 --from-bp 155235878 --to-bp 155235878 \
--covar masterfile_updated_GP2_v4_covariateFile_wAGE_FEB2023.txt \
--covar-name SEX,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 \
--out dominant_AAC --ci 0.95

# CHROM	POS	ID	REF	ALT	A1	TEST	OBS_CT	OR	SE	L95	U95	T_STAT	P
# 1	155235878	chr1:155235878:G:T	T	G	G	DOM	1323	1.96795	0.171179	1.40704	2.75247	3.9549	7.65654e-05

cd GP2-v4-AAC

plink2 --bfile GP2-v4-AAC-noRelateds \
--glm --chr 1 --from-bp 155235878 --to-bp 155235878 \
--covar masterfile_updated_GP2_v4_covariateFile_wAGE_FEB2023.txt \
--covar-name SEX,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 --out additive_AAC --ci 0.95

# CHROM	POS	ID	REF	ALT	A1	TEST	OBS_CT	OR	SE	L95	U95	T_STAT	P
# 1	155235878	chr1:155235878:G:T	T	G	G	ADD	1323	1.95981	0.146109	1.47179	2.60964	4.60511	4.12248e-06

cd GP2-v4-AAC

plink2 --bfile GP2-v4-AAC-noRelateds --glm recessive --chr 1 --from-bp 155235878 --to-bp 155235878 \
--covar masterfile_updated_GP2_v4_covariateFile_wAGE_FEB2023.txt \
--covar-name SEX,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 \
--out recessive_AAC --ci 0.95

# CHROM	POS	ID	REF	ALT	A1	TEST	OBS_CT	OR	SE	L95	U95	T_STAT	P
# 1	155235878	chr1:155235878:G:T	T	G	G	REC	1323	4.38998	0.40664	1.97847	9.74083	3.63792	0.000274846
```

```bash
####### AFR #######

cd AFR/GP2-v4-AFR-wNIGERIAN-NB

plink2 --bfile GP2-v4-AFR-wNIGERIAN-NB-noRelateds \
--glm dominant --chr 1 --from-bp 155235878 --to-bp 155235878 \
--covar masterfile_updated_GP2_v4_covariateFile_wAGE_FEB2023.txt \
--covar-name SEX,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 \
--out dominant_AFR --ci 0.95

# CHROM	POS	ID	REF	ALT	A1	TEST	OBS_CT	OR	SE	L95	U95	T_STAT	P
# 1	155235878	chr1:155235878:G:T	T	G	G	DOM	1673	1.74201	0.108915	1.40716	2.15655	5.09611	3.46711e-07

cd AFR/GP2-v4-AFR-wNIGERIAN-NB

plink2 --bfile GP2-v4-AFR-wNIGERIAN-NB-noRelateds \
--glm --chr 1 --from-bp 155235878 --to-bp 155235878 --covar masterfile_updated_GP2_v4_covariateFile_wAGE_FEB2023.txt \
--covar-name SEX,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 \
--out additive_AFR --ci 0.95

# CHROM	POS	ID	REF	ALT	A1	TEST	OBS_CT	OR	SE	L95	U95	T_STAT	P
# 1	155235878	chr1:155235878:G:T	T	G	G	ADD	1673	1.75163	0.0873806	1.47592	2.07884	6.41499	1.40836e-10

cd AFR/GP2-v4-AFR-wNIGERIAN-NB

plink2 --bfile GP2-v4-AFR-wNIGERIAN-NB-noRelateds \
--glm recessive --chr 1 --from-bp 155235878 --to-bp 155235878 --covar masterfile_updated_GP2_v4_covariateFile_wAGE_FEB2023.txt \
--covar-name SEX,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 \
--out recessive_AFR --ci 0.95

# CHROM	POS	ID	REF	ALT	A1	TEST	OBS_CT	OR	SE	L95	U95	T_STAT	P
# 1	155235878	chr1:155235878:G:T	T	G	G	REC	1673	3.46287	0.223441	2.23482	5.36573	5.55895	2.71399e-08
```

```bash
### 1_155235878 AAO analyses in the AAC and AFR cohorts + meta-analysis

cd AAC/GP2-v4-AAC

plink --bfile Cases_GP2-v4-AAC-noRelateds \
--chr 1 --covar masterfile_updated_GP2_v4_covariateFile_wAGE_FEB2023.txt \
--covar-name SEX,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 \
--from-bp 155235878 --to-bp 155235878 \
--linear --out linear_regression \
--pheno AFR/GP2-v4-AFR-wNIGERIAN-NB/age_cases.txt

# cat linear_regression.assoc.linear
#  CHR                  SNP         BP   A1       TEST    NMISS       BETA         STAT            P 
#    1   chr1:155235878:G:T  155235878    G        ADD      183     -4.156       -2.443      0.01559
#    1   chr1:155235878:G:T  155235878    G        SEX      183     0.6536       0.3176       0.7512
#    1   chr1:155235878:G:T  155235878    G        PC1      183    0.02188       0.1637       0.8701
#    1   chr1:155235878:G:T  155235878    G        PC2      183    -0.1008      -0.1855        0.853
#    1   chr1:155235878:G:T  155235878    G        PC3      183     -0.256      -0.7346       0.4636
#    1   chr1:155235878:G:T  155235878    G        PC4      183    -0.7727       -1.502        0.135
#    1   chr1:155235878:G:T  155235878    G        PC5      183     0.2459       0.5869        0.558
#    1   chr1:155235878:G:T  155235878    G        PC6      183     -1.119       -1.735      0.08456
#    1   chr1:155235878:G:T  155235878    G        PC7      183    -0.7827       -1.549       0.1232
#    1   chr1:155235878:G:T  155235878    G        PC8      183     0.2335       0.5359       0.5927
#    1   chr1:155235878:G:T  155235878    G        PC9      183     0.3095       0.5127       0.6088
#    1   chr1:155235878:G:T  155235878    G       PC10      183    -0.7813       -1.406       0.1615

cd AFR/GP2-v4-AFR-wNIGERIAN-NB

plink --bfile Cases_GP2-v4-AFR-wNIGERIAN-NB-noRelateds \
--chr 1 --covar masterfile_updated_GP2_v4_covariateFile_wAGE_FEB2023.txt \
--covar-name SEX,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 \
--from-bp 155235878 --to-bp 155235878 \
--linear --out linear_regression \
--pheno age_cases.txt 

# cat linear_regression.assoc.linear
#  CHR                  SNP         BP   A1       TEST    NMISS       BETA         STAT            P 
#    1   chr1:155235878:G:T  155235878    G        ADD      687     -2.004       -3.483    0.0005284
#    1   chr1:155235878:G:T  155235878    G        SEX      687    -0.6066      -0.6941       0.4878
#    1   chr1:155235878:G:T  155235878    G        PC1      687    -0.1977       -1.083        0.279
#    1   chr1:155235878:G:T  155235878    G        PC2      687     0.1228       0.4397       0.6603
#    1   chr1:155235878:G:T  155235878    G        PC3      687    -0.1012      -0.3755       0.7074
#    1   chr1:155235878:G:T  155235878    G        PC4      687    -0.3303       -1.219       0.2233
#    1   chr1:155235878:G:T  155235878    G        PC5      687    -0.5364       -2.115      0.03476
#    1   chr1:155235878:G:T  155235878    G        PC6      687    -0.3715       -1.391       0.1646
#    1   chr1:155235878:G:T  155235878    G        PC7      687     -0.282       -1.178       0.2393
#    1   chr1:155235878:G:T  155235878    G        PC8      687    -0.2463       -1.448       0.1481
#    1   chr1:155235878:G:T  155235878    G        PC9      687     0.2625        1.022        0.307
#    1   chr1:155235878:G:T  155235878    G       PC10      687     0.3814        1.905      0.05719
```

```R
install.packages("rmeta")
library(rmeta)
library(data.table)
sumstats <- fread("summary.txt", header = T)
met <- meta.summaries(d = sumstats$BETA, se = sumstats$SE, method = c("fixed"), logscale = F, names = sumstats$COHORT) # logscale option set for OR.

met$test # gives you the Z and P for the meta-analysis.
met

## Make summary.txt table
# BETA	SE	P	COHORT
# -2.004	0.57	0.0005	African
# -4.156	0.58	0.015	AfricanAdmixed

# Fixed-effects meta-analysis
# Call: meta.summaries(d = sumstats$BETA, se = sumstats$SE, method = c("fixed"), 
#     logscale = F, names = sumstats$COHORT)
# Summary effect=-3.06   95% CI (-3.86, -2.26)
# Estimated heterogeneity variance: 2  p= 0.008 
```

# WGS Annotation
For T2

```bash
## Annotate GBA in WGS data (206 samples)
cd /data/CARD_training
module load annovar
## test.vcf contains WGS data only for GBA 

table_annovar.pl test.vcf $ANNOVAR_DATA/hg38 -buildver hg38 \
--thread 16 \
--out ALL.annovar \
-arg '-splicing 15',,, \
-remove \
-protocol refGene,avsnp147,ljb26_all,gnomad312_genome \
-operation g,f,f,f \
-nastring . \
--vcfinput

## Now extract carriers for coding variants 

plink --vcf test.vcf --chr 1 --from-bp 155236249 --to-bp 155236249 --recode A --pheno pheno.txt --out p.I320S --double-id
plink --vcf test.vcf --chr 1 --from-bp 155236269 --to-bp 155236269 --recode A --pheno pheno.txt --out p.M313I --double-id
plink --vcf test.vcf --chr 1 --from-bp 155236377 --to-bp 155236377 --recode A --pheno pheno.txt --out p.G277G --double-id
plink --vcf test.vcf --chr 1 --from-bp 155238228 --to-bp 155238228 --recode A --pheno pheno.txt --out p.W136R --double-id
plink --vcf test.vcf --chr 1 --from-bp 155239962 --to-bp 155239962 --recode A --pheno pheno.txt --out p.S77R --double-id
plink --vcf test.vcf --chr 1 --from-bp 155240707 --to-bp 155240707 --recode A --pheno pheno.txt --out p.K13R --double-id

## Confirm top hit carriers

plink --vcf test.vcf --chr 1 --from-bp 155235878 --to-bp 155235878 --recode A --pheno pheno.txt --out tophit_155235878 --double-id
```

# Conditional Analysis

```bash
###### CONDITIONAL ANALYSES on the top two SNPs ###### 

## NIGERIA 2

cat list | while read LINE 
do
echo $LINE
plink2 --pfile chr1_AFRICAN --extract range GBA_SNPS.txt --make-bed --out $LINE.AFRICAN
done

plink --bfile 1_155869005.AFRICAN --merge-list list_SNPS.txt --make-bed --out ALL_35SNPS.AFRICAN

plink --bfile ALL_35SNPS.AFRICAN --pheno pheno_AFRICA.txt --update-sex sex_AFRICA.txt --recode A --out ALL_35SNPS.AFRICAN_pheno_sex

## AAC

cat list | while read LINE 
do
echo $LINE
plink2 --pfile chr1_AAC --extract range GBA_SNPS.txt --make-bed --out $LINE.AAC
done

plink --bfile 1_155869005.AAC --merge-list list_SNPS.txt --recode A --out ALL_35SNPS.AAC

## REFORMAT

sed "s/:/_/g" ALL_35SNPS.AAC.raw > ALL_35SNPS.AAC_reformatted.raw
grep -v "IPDGCAF" ALL_35SNPS.AAC_reformatted.raw > ALL_35SNPS.AAC_reformatted_final.raw

sed "s/:/_/g" ALL_35SNPS.AFRICAN_pheno_sex.raw > ALL_35SNPS.AFRICAN_reformatted.raw

## CHECK R2
```

```R
# Data Munging

library(data.table)
cov_df <- fread("covariate_file_wAGE.txt", header = T)
snps_df <- fread("ALL_35SNPS.AFRICAN_reformatted.raw", header = T)
merged_df <- merge(cov_df, snps_df, by="IID")
summary(merged_df)
merged_df$CASE = merged_df$PHENO - 1
merged_df$female = merged_df$SEX.x - 1
merged_df$dataset = "AFR"

# Model parameters

snp_params <- paste(names(snps_df)[7:59], collapse=' + ')
other_params <- "CASE ~ female"
pc_params <- "PC1 + PC2 + PC3 + PC4 + PC5"
model_formula <- paste(other_params, snp_params, pc_params, sep =" + ")

# Build model

conditional_model <- glm(model_formula, data = merged_df, family="binomial")
summary(conditional_model)

# Data Munging

library(data.table)
cov_df <- fread("covariate_file_wAGE.txt", header = T)
snps_df <- fread("ALL_35SNPS.AAC_reformatted.raw", header = T)
merged_df <- merge(cov_df, snps_df, by="IID")
summary(merged_df)
merged_df$CASE = merged_df$PHENO - 1
merged_df$female = merged_df$SEX.x - 1
merged_df$dataset = "AFR"

# Model parameters

snp_params <- paste(names(snps_df)[7:59], collapse=' + ')
other_params <- "CASE ~ female"
pc_params <- "PC1 + PC2 + PC3 + PC4 + PC5"
model_formula <- paste(other_params, snp_params, pc_params, sep =" + ")

# Build model

conditional_model <- glm(model_formula, data = merged_df, family="binomial")
summary(conditional_model)
```

# Miami plot compared to Europeans

```bash
cd Miami_Plot_equal_sample

## METAL AFR + AAC data without 23andMe Sum stats = 1,200 cases + 2,445 controls

# UNCOMMENT THE NEXT LINE TO ENABLE GenomicControl CORRECTION
SCHEME STDERR
GENOMICCONTROL ON

# === DESCRIBE AND PROCESS THE FIRST INPUT FILE ===
MARKER markerID
ALLELE effect_allele other_allele
# FREQ   maf
EFFECT beta
STDERR LOG(OR)_SE
PVALUE P
# WEIGHT OBS_CT 
PROCESS GP2-v4-AAC-GWAS-MAF005-FEB2023.UpdatedforMETAL.tab

# === DESCRIBE AND PROCESS THE SECOND INPUT FILE ===
MARKER MarkerName
ALLELE Allele1	Allele2
# FREQ   maf
EFFECT Effect
STDERR StdErr
PVALUE P-value
# WEIGHT N
PROCESS AFR-ONLY-META-UpdatedforMETAL1.tbl


OUTFILE MY_TWO_GWAS_FILES_META.tbl
ANALYZE HETEROGENEITY

QUIT
```

```R
## Format sum stats splitting chr and pos

library(data.table)
library(dplyr)
library(tidyr)
library(plyr)
META_AFR_AAC <- fread("METAANALYSIS1.TBL", header = T)
META_AFR_AAC$CHR <- ldply(strsplit(as.character(META_AFR_AAC$MarkerName), split = ":"))[[1]]
META_AFR_AAC$BP <- ldply(strsplit(as.character(META_AFR_AAC$MarkerName), split = ":"))[[2]]
write.table(META_AFR_AAC, file = "chr_pos_METAANALYSIS1.TBL", quote = F, row.names = F, sep = "\t")

```

```bash 
sed 's/chr//g' chr_pos_METAANALYSIS1.TBL > formatted_chr_pos_METAANALYSIS1.TBL
```

```bash
## EUR GWAS at similar N cases and controls

## Randomly select cases
cd /data/CARD/PD/GP2/genotypes/GP2/round3/imputed/EUR

module load plink/2.0_alpha_1_final
plink2 --pfile /data/CARD/PD/GP2/genotypes/GP2/round3/imputed/EUR/chr9_EUR_prune_final \
--make-bed --out chr9_EUR_prune_final

module load plink/1.9
plink --bfile chr9_EUR_prune_final \
--filter-cases --out cases --make-bed

## Here we have 7,409 cases and we want to keep 1,200 
```

```R

library(data.table)
fam <- fread("cases.fam", header = F)
colnames(fam) <- c("FID", "IID", "MAT", "PAT", "SEX", "PHENO")
fam_subsetted1 <- fam[sample(1:nrow(fam), 1200), ]
write.table(fam_subsetted1, "EUR_tokeep_cases.txt", quote = F, sep = "\t", row.names = F, col.names = F)
```

```bash
## Randomly select controls

module load plink/1.9
plink --bfile chr9_EUR_prune_final --filter-controls --out controls --make-bed

## Here we have 4,277 controls and we want to keep 2,445 
```

```R

library(data.table)
fam <- fread("controls.fam", header = F)
colnames(fam) <- c("FID", "IID", "MAT", "PAT", "SEX", "PHENO")
fam_subsetted1 <- fam[sample(1:nrow(fam), 2445), ]
write.table(fam_subsetted1, "EUR_tokeep_controls.txt", quote = F, sep = "\t", row.names = F, col.names = F)
```

```bash
## Merge list of cases and controls
cat EUR_tokeep_cases.txt EUR_tokeep_controls.txt > ALL_tokeep.txt
awk '{print $1, $2}' ALL_tokeep.txt> IDs_ALL_tokeep.txt
ALL_tokeep.txt IDs_formatted_to_keep.txt > pheno_test.txt
awk '{print $7, $8, $6}' pheno_test.txt > pheno.txt

## Extract cases and controls and run GWAS
module load plink/2.0_alpha_1_final
cat list.txt | while read LINE
do
echo $LINE
plink2 --pfile ${LINE}_EUR \
--keep IDs_formatted_to_keep.txt \
--pheno pheno.txt --make-bed \
--out Miami_Plot_equal_sample/subsetted${LINE}
done 

module load plink/1.9
cat list.txt | while read LINE
do
echo $LINE
plink --bfile Miami_Plot_equal_sample/subsetted${LINE} \
--allow-no-sex \
--logistic --out Miami_Plot_equal_sample/sampled$LINE
done 

cat sampledchr*.assoc.logistic > ALL_EUR_assoc.logistic
grep -v "TEST" ALL_EUR_assoc.logistic > test_ALL_EUR_assoc.logistic
grep -v "NA" test_ALL_EUR_assoc.logistic > no_NA_test_ALL_EUR_assoc.logistic
```

```R
library(data.table)
stats <- fread("no_NA_test_ALL_EUR_assoc.logistic", header = F)
colnames(stats) <- c("CHR","SNP","BP","A1","TEST","NMISS","OR","STAT","P")
subset(stats, P < 1E-100)
write.table(stats_revised, "ALL_EUR_assoc_header.logistic", quote = F, sep = "\t", row.names = F, col.names = T)
```

```python
## Miami_plot
import gwaslab as gl
import pandas as pd

mysumstats = gl.plot_miami(path1="formatted_chr_pos_METAANALYSIS1.TBL",
						   path2="ALL_EUR_assoc_header.logistic",
                           cols1=["CHR","BP","P-value"],
                           cols2=["CHR","BP","P"],
                           titles=["African & African Admixed GWAS meta-analysis", "European GWAS meta-analysis"],
                           titles_pad=[0.15,0.0],
                           cut=20,
                           region_grid=True,
                           save="Miami_sampled.jpg",
                           saveargs={"dpi":400,"facecolor":"white"}
                           )
```

# GBA Gauchian Caller

```wdl
version 1.0
# WORKFLOW DEFINITION 

workflow CallGBAvariants {
  input {
    File cram
    File cram_index
    File ref_fasta
    File ref_fasta_index
    String sample_id
    String? gauch_docker_override
    String gauch_docker = select_first([gauch_docker_override, "avalillarionova/gauchian:1.0.2"])
    Int? preemptible_attempts_override
    Int preemptible_attempts = select_first([preemptible_attempts_override, "2"])
    String runtime_zones
    Int threads
    }

  call Gauchian {
      input:
        cram = cram,
        cram_index= cram_index,
        ref_fasta = ref_fasta,
        ref_fasta_index = ref_fasta_index,
        sample_id = sample_id,
        gauch_docker = gauch_docker,
	    threads = threads,
        preemptible_attempts = preemptible_attempts,
        runtime_zones = runtime_zones
    }
      # Outputs that will be retained when execution is complete
  output {
      File gauchian_tsv = Gauchian.output_tsv
      File gauchian_json = Gauchian.output_json
  }

}

# TASK DEFINITIONS

task Gauchian {
  input {
  # Command parameters
    File cram
    File cram_index
    File ref_fasta
    File ref_fasta_index
    String sample_id

  # Runtime parameters
    Int additional_disk_space_gb = 10
    Int? machine_mem_gb
    Int preemptible_attempts
    String gauch_docker
    String runtime_zones
    Int threads  
    Int disk_space_gb = ceil(2*size(cram, "G") + size(ref_fasta, "G")) + additional_disk_space_gb
  }

  command {  
  echo "~{cram}" > manifest_gauch.txt
  python -m gauchian --manifest manifest_gauch.txt \
                   --genome 38 \
                   --reference ~{ref_fasta} \
                   --prefix ~{sample_id} \
                   --outDir ~{sample_id} \
                   --threads ~{threads}

  mv ~{sample_id}/*.tsv "~{sample_id}.tsv"
  mv ~{sample_id}/*.json "~{sample_id}.json"
  }
  runtime {
    docker: gauch_docker
    memory: select_first([machine_mem_gb, 10]) + " GB"
    cpu: threads
    disks: "local-disk " + disk_space_gb + " HDD"
    preemptible: preemptible_attempts
    zones: runtime_zones
  }
  output {
    File output_tsv = "~{sample_id}.tsv"
    File output_json = "~{sample_id}.json"
  }
}

```